In [1]:
#import dependencies 
import pandas as pd 
import requests 
from sqlalchemy import create_engine
import psycopg2
import config 

In [2]:
#connect to postgres database 
conn = psycopg2.connect(database="donniedata", user=config.db_user, password=config.db_password, sslmode="disable")
cur = conn.cursor();
conn.autocommit = True

In [3]:
#get sample of data -1000 rows
endpoint = "https://data.sfgov.org/resource/imvp-dq3v.json?$limit=1000"
r = requests.get(url=endpoint)

In [4]:
#convert to datframe 
data_df = pd.DataFrame(r.json())

In [5]:
# review enforce data types 
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   transmission_datetime  1000 non-null   object
 1   post_id                1000 non-null   object
 2   street_block           1000 non-null   object
 3   payment_type           1000 non-null   object
 4   session_start_dt       1000 non-null   object
 5   session_end_dt         1000 non-null   object
 6   meter_event_type       1000 non-null   object
 7   gross_paid_amt         1000 non-null   object
dtypes: object(8)
memory usage: 62.6+ KB


In [6]:
data_df.head()

,transmission_datetime,post_id,street_block,payment_type,session_start_dt,session_end_dt,meter_event_type,gross_paid_amt
0,196158860_4_06092020073616,700-14150,VALENCIA ST 1400,CASH,2020-06-09T07:36:16.000,2020-06-09T10:36:00.000,NS,0.8
1,196158861_4_06092020073722,471-07180,HARRISON ST 700,CREDIT CARD,2020-06-09T07:37:22.000,2020-06-09T17:30:10.000,NS,4.25
2,196158862_4_06092020073733,524-01320,KING ST 100,CREDIT CARD,2020-06-09T07:37:33.000,2020-06-09T17:00:09.000,NS,4
3,196158863_4_06092020073706,651-03060,SANSOME ST 300,CASH,2020-06-09T07:37:06.000,2020-06-09T10:58:16.000,AT,1.35
4,196158864_4_06092020073759,359-09080,CLAY ST 900,CREDIT CARD,2020-06-09T07:37:59.000,2020-06-09T15:00:10.000,NS,3


In [27]:
#get unique values of dimension from current data pullto further reduce checking each value 
streets_unique = data_df.street_block.unique()
streets_unique[0:10]

array(['GRANT AVE 1100', 'PACIFIC AVE 400', 'NEW MONTGOMERY ST 100',
       'HOWARD ST 100', 'CLAY ST 300', '07TH ST 100',
       'SAINT JOSEPHS AVE 300', 'HOWARD ST 500', 'MISSION ST 0',
       'MAIN ST 100'], dtype=object)

In [11]:

street_list = data_df.street_block.to_list()

In [55]:
#first test with all values to ensure script only inserts values not already in table 
#convert to tuples for insertion 
street_list = list(data_df[['street_block']].to_records(index=False))
street_list[0:5]

[('VALENCIA ST 1400',),
 ('HARRISON ST 700',),
 ('KING ST 100',),
 ('SANSOME ST 300',),
 ('CLAY ST 900',)]

In [60]:
street_list[0]

('VALENCIA ST 1400',)

In [59]:
sql = '''
DO $$
BEGIN 
IF NOT EXISTS (select 1 from sf_ticket_trans.dim_street where street_block = %s) 
THEN INSERT INTO sf_ticket_trans.dim_street (street_block) VALUES (%s);
END IF;
END;
$$ 

'''
cur.executemany(sql, street_list)

IndexError: invalid index (1)

In [ ]:
cur.executemany()

In [46]:
print(len(street_list))

1000
